In [15]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torchvision import transforms

In [16]:
train_data_path = '/opt/ml/input/data/train'
df_original = pd.read_csv(os.path.join(train_data_path, 'train.csv'))
df = df_original.copy()

In [17]:
del df['race']
del df['id']
df['gender'] = df['gender'].map({'male':0, 'female':1})
bins = [0, 29, 59, 100]
labels = [0, 1, 2]
df['age'] = pd.cut(df['age'], bins=bins, labels=labels)

In [18]:
df['age'].value_counts()

0    1281
1    1227
2     192
Name: age, dtype: int64

In [19]:
def label(row):
    return row['age'] + 3*row['gender']

df['label'] = df.apply(label, axis=1)
df['mask'] = [['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal'] for _ in range(len(df))]

In [20]:
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df.label)

In [14]:
df_train

,gender,age,path,label,mask
1061,1,1,001975_female_Asian_57,4,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
638,1,1,001313_female_Asian_45,4,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
35,1,1,000044_female_Asian_58,4,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
70,0,1,000229_male_Asian_56,1,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
576,0,0,001221_male_Asian_19,0,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
...,...,...,...,...,...
187,1,1,000570_female_Asian_54,4,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
725,0,1,001457_male_Asian_59,1,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
1020,0,0,001827_male_Asian_18,0,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."
212,1,1,000607_female_Asian_53,4,"[incorrect_mask, mask1, mask2, mask3, mask4, m..."


In [ ]:
def generate_mask_field(df):
    
    def _get_extension(row):
        for f in os.listdir(os.path.join(train_data_path, 'images', row.path)):
            if f.startswith(row['mask']):
                return os.path.join(train_data_path, 'images', row.path, f)
        raise FileNotFoundError

    def _label(row):
        return row['age'] + 3*row['gender'] + 6*row['mask']
    
    mask_map = {'incorrect_mask': 1, 'mask1': 0, 'mask2': 0, 'mask3': 0,
                'mask4': 0, 'mask5': 0, 'normal': 2}    

    df = df.explode('mask')
    df['path'] = df.apply(_get_extension, axis=1)
    df['mask'] = df['mask'].map(mask_map)
    df['label'] = df.apply(_label, axis=1)

    return df

In [ ]:
df.to_csv('df_all.csv', index=False)

In [ ]:
df_train = generate_mask_field(df_train)
df_valid = generate_mask_field(df_valid)

In [33]:
df_train.to_csv('df_train.csv', index=False)
df_valid.to_csv('df_valid.csv', index=False)